In [204]:
# module imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import random
from sklearn.decomposition import PCA

# model imports
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import KernelPCA


# processing imports
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import matthews_corrcoef, precision_score, recall_score

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, balanced_accuracy_score,classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

np.random.seed(1635848)

In [205]:
# fetch the training file
file_path_full_training_set = '/kaggle/input/kdd-data/KDDTrain.txt'
file_path_test = '/kaggle/input/kdd-data/KDDTest.txt'

In [206]:
df = pd.read_csv(file_path_full_training_set)
test_df = pd.read_csv(file_path_test)

In [207]:
columns = (['duration'
,'protocol_type'
,'service'
,'flag'
,'src_bytes'
,'dst_bytes'
,'land'
,'wrong_fragment'
,'urgent'
,'hot'
,'num_failed_logins'
,'logged_in'
,'num_compromised'
,'root_shell'
,'su_attempted'
,'num_root'
,'num_file_creations'
,'num_shells'
,'num_access_files'
,'num_outbound_cmds'
,'is_host_login'
,'is_guest_login'
,'count'
,'srv_count'
,'serror_rate'
,'srv_serror_rate'
,'rerror_rate'
,'srv_rerror_rate'
,'same_srv_rate'
,'diff_srv_rate'
,'srv_diff_host_rate'
,'dst_host_count'
,'dst_host_srv_count'
,'dst_host_same_srv_rate'
,'dst_host_diff_srv_rate'
,'dst_host_same_src_port_rate'
,'dst_host_srv_diff_host_rate'
,'dst_host_serror_rate'
,'dst_host_srv_serror_rate'
,'dst_host_rerror_rate'
,'dst_host_srv_rerror_rate'
,'attack'
,'level'])

In [208]:
df.columns = columns
test_df.columns = columns

In [209]:
# Categorical features analysis
print('Training set categorical features:')
for col_name in df.columns:
    if df[col_name].dtypes == 'object' :
        unique_cat = len(df[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} categories".format(col_name=col_name, unique_cat=unique_cat))

Training set categorical features:
Feature 'protocol_type' has 3 categories
Feature 'service' has 70 categories
Feature 'flag' has 11 categories
Feature 'attack' has 23 categories


In [210]:
# colums that are categorical and not binary yet: protocol_type (column 2), service (column 3), flag (column 4).
# explore categorical features
print('Training set:')
for col_name in df.columns:
    if df[col_name].dtypes == 'object' :
        unique_cat = len(df[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} categories".format(col_name=col_name, unique_cat=unique_cat))

Training set:
Feature 'protocol_type' has 3 categories
Feature 'service' has 70 categories
Feature 'flag' has 11 categories
Feature 'attack' has 23 categories


In [211]:
print('Distribution of categories in service:')
print(df['service'].value_counts().sort_values(ascending=False).head())

Distribution of categories in service:
service
http        40338
private     21853
domain_u     9043
smtp         7313
ftp_data     6859
Name: count, dtype: int64


In [212]:
# Test set
print('Test set:')
for col_name in test_df.columns:
    if test_df[col_name].dtypes == 'object' :
        unique_cat = len(test_df[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} categories".format(col_name=col_name, unique_cat=unique_cat))

Test set:
Feature 'protocol_type' has 3 categories
Feature 'service' has 64 categories
Feature 'flag' has 11 categories
Feature 'attack' has 38 categories


In [213]:
# Attack Distribution Comparison Analysis
print("="*60)
print("ATTACK TYPE DISTRIBUTION ANALYSIS")
print("="*60)

print("\nTraining Set Attack Distribution:")
print(df['attack'].value_counts().head(10))

print("\nTest Set Attack Distribution:")
print(test_df['attack'].value_counts().head(10))

print("\nUnique Attacks Comparison:")
train_attacks = set(df['attack'].unique())
test_attacks = set(test_df['attack'].unique())

print(f"Training attacks: {len(train_attacks)} unique types")
print(f"Test attacks: {len(test_attacks)} unique types")

attacks_only_in_train = train_attacks - test_attacks
attacks_only_in_test = test_attacks - train_attacks
common_attacks = train_attacks.intersection(test_attacks)

print(f"\nAttacks ONLY in training: {len(attacks_only_in_train)}")
if attacks_only_in_train:
    print(f"   {list(attacks_only_in_train)}")

print(f"\nAttacks ONLY in test: {len(attacks_only_in_test)}")  
if attacks_only_in_test:
    print(f"   {list(attacks_only_in_test)}")

print(f"\nCommon attacks: {len(common_attacks)}")

ATTACK TYPE DISTRIBUTION ANALYSIS

Training Set Attack Distribution:
attack
normal         67342
neptune        41214
satan           3633
ipsweep         3599
portsweep       2931
smurf           2646
nmap            1493
back             956
teardrop         892
warezclient      890
Name: count, dtype: int64

Test Set Attack Distribution:
attack
normal          9711
neptune         4656
guess_passwd    1231
mscan            996
warezmaster      944
apache2          737
satan            735
processtable     685
smurf            665
back             359
Name: count, dtype: int64

Unique Attacks Comparison:
Training attacks: 23 unique types
Test attacks: 38 unique types

Attacks ONLY in training: 2
   ['warezclient', 'spy']

Attacks ONLY in test: 17
   ['processtable', 'ps', 'apache2', 'xterm', 'mailbomb', 'mscan', 'udpstorm', 'snmpguess', 'xlock', 'worm', 'saint', 'sendmail', 'named', 'httptunnel', 'xsnoop', 'snmpgetattack', 'sqlattack']

Common attacks: 21


In [214]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Define categorical columns for encoding
categorical_columns = ['protocol_type', 'service', 'flag']

# Extract categorical values
df_categorical_values = df[categorical_columns]
testdf_categorical_values = test_df[categorical_columns]
df_categorical_values.head()

,protocol_type,service,flag
0,udp,other,SF
1,tcp,private,S0
2,tcp,http,SF
3,tcp,http,SF
4,tcp,private,REJ


In [215]:
# Generate column names for categorical encoding
unique_protocol=sorted(df.protocol_type.unique())
string1 = 'Protocol_type_'
unique_protocol2=[string1 + x for x in unique_protocol]

# Use combined unique services from both train and test sets
unique_service_train = set(df.service.unique())
unique_service_test = set(test_df.service.unique())
unique_service_combined = sorted(list(unique_service_train.union(unique_service_test)))
string2 = 'service_'
unique_service2 = [string2 + x for x in unique_service_combined]

# Flag columns
unique_flag=sorted(df.flag.unique())
string3 = 'flag_'
unique_flag2=[string3 + x for x in unique_flag]

# Combined columns for consistent encoding
dumcols = unique_protocol2 + unique_service2 + unique_flag2
testdumcols = dumcols

print(f"Total columns: {len(dumcols)}")
print(f"Protocol columns: {len(unique_protocol2)}")  
print(f"Service columns: {len(unique_service2)}")
print(f"Flag columns: {len(unique_flag2)}")

print(f"\nServices only in train: {unique_service_train - unique_service_test}")
print(f"Services only in test: {unique_service_test - unique_service_train}")
print(f"Combined services: {len(unique_service_combined)}")

Total columns: 84
Protocol columns: 3
Service columns: 70
Flag columns: 11

Services only in train: {'harvest', 'http_2784', 'http_8001', 'urh_i', 'aol', 'red_i'}
Services only in test: set()
Combined services: 70


In [216]:
df_categorical_values_enc=df_categorical_values.apply(LabelEncoder().fit_transform)
print(df_categorical_values_enc.head())
# test set
testdf_categorical_values_enc=testdf_categorical_values.apply(LabelEncoder().fit_transform)

   protocol_type  service  flag
0              2       44     9
1              1       49     5
2              1       24     9
3              1       24     9
4              1       49     1


In [217]:
# OneHot encoding with proper handling of different categorical values
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Combine train and test categorical data to capture all possible categories
combined_categorical = pd.concat([df_categorical_values_enc, testdf_categorical_values_enc], ignore_index=True)

# Fit encoder on combined data to handle all categories consistently
enc = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
enc.fit(combined_categorical)

# Transform train and test sets
df_categorical_values_encenc = enc.transform(df_categorical_values_enc)
df_cat_data = pd.DataFrame(df_categorical_values_encenc, columns=dumcols)

testdf_categorical_values_encenc = enc.transform(testdf_categorical_values_enc)
testdf_cat_data = pd.DataFrame(testdf_categorical_values_encenc, columns=dumcols)

print(f"Train set categorical shape: {df_cat_data.shape}")
print(f"Test set categorical shape: {testdf_cat_data.shape}")
print(f"Column names match: {list(df_cat_data.columns) == list(testdf_cat_data.columns)}")

df_cat_data.head()

Train set categorical shape: (125972, 84)
Test set categorical shape: (22543, 84)
Column names match: True


,Protocol_type_icmp,Protocol_type_tcp,Protocol_type_udp,service_IRC,service_X11,service_Z39_50,service_aol,service_auth,service_bgp,service_courier,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [218]:
# Verify column structure consistency
print("Column structure verification:")
print(f"Train categorical shape: {df_cat_data.shape}")
print(f"Test categorical shape: {testdf_cat_data.shape}")
print(f"Columns match: {df_cat_data.shape[1] == testdf_cat_data.shape[1]}")

Column structure verification:
Train categorical shape: (125972, 84)
Test categorical shape: (22543, 84)
Columns match: True


In [219]:
# Final verification of column structure consistency
print("Column structure consistency achieved")

print(f"Final train categorical shape: {df_cat_data.shape}")
print(f"Final test categorical shape: {testdf_cat_data.shape}")
assert df_cat_data.shape[1] == testdf_cat_data.shape[1], "Column count mismatch!"
print("Verification complete - ready to proceed")

Column structure consistency achieved
Final train categorical shape: (125972, 84)
Final test categorical shape: (22543, 84)
Verification complete - ready to proceed


In [220]:
newdf=df.join(df_cat_data)
newdf.drop('flag', axis=1, inplace=True)
newdf.drop('protocol_type', axis=1, inplace=True)
newdf.drop('service', axis=1, inplace=True)
# test data
newdf_test=test_df.join(testdf_cat_data)
newdf_test.drop('flag', axis=1, inplace=True)
newdf_test.drop('protocol_type', axis=1, inplace=True)
newdf_test.drop('service', axis=1, inplace=True)
print(newdf.shape)
print(newdf_test.shape)
newdf=df.join(df_cat_data)
newdf.drop('flag', axis=1, inplace=True)
newdf.drop('protocol_type', axis=1, inplace=True)
newdf.drop('service', axis=1, inplace=True)
# test data
newdf_test=test_df.join(testdf_cat_data)
newdf_test.drop('flag', axis=1, inplace=True)
newdf_test.drop('protocol_type', axis=1, inplace=True)
newdf_test.drop('service', axis=1, inplace=True)
print(newdf.shape)
print(newdf_test.shape)

(125972, 124)
(22543, 124)
(125972, 124)
(22543, 124)


In [221]:
# take label column
labeldf=newdf['attack']
labeldf_test=newdf_test['attack']
# change the label column
newlabeldf=labeldf.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                           'ipsweep' : 2,'nmap' : 2,'portsweep' : 2,'satan' : 2,'mscan' : 2,'saint' : 2
                           ,'ftp_write': 3,'guess_passwd': 3,'imap': 3,'multihop': 3,'phf': 3,'spy': 3,'warezclient': 3,'warezmaster': 3,'sendmail': 3,'named': 3,'snmpgetattack': 3,'snmpguess': 3,'xlock': 3,'xsnoop': 3,'httptunnel': 3,
                           'buffer_overflow': 4,'loadmodule': 4,'perl': 4,'rootkit': 4,'ps': 4,'sqlattack': 4,'xterm': 4})
newlabeldf_test=labeldf_test.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                           'ipsweep' : 2,'nmap' : 2,'portsweep' : 2,'satan' : 2,'mscan' : 2,'saint' : 2
                           ,'ftp_write': 3,'guess_passwd': 3,'imap': 3,'multihop': 3,'phf': 3,'spy': 3,'warezclient': 3,'warezmaster': 3,'sendmail': 3,'named': 3,'snmpgetattack': 3,'snmpguess': 3,'xlock': 3,'xsnoop': 3,'httptunnel': 3,
                           'buffer_overflow': 4,'loadmodule': 4,'perl': 4,'rootkit': 4,'ps': 4,'sqlattack': 4,'xterm': 4})
# put the new label column back
newdf['attack'] = newlabeldf
newdf_test['attack'] = newlabeldf_test
print(newdf['attack'].head())


0    0
1    1
2    0
3    0
4    1
Name: attack, dtype: int64


/tmp/ipykernel_36/1623462165.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  newlabeldf=labeldf.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
/tmp/ipykernel_36/1623462165.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  newlabeldf_test=labeldf_test.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm'

In [222]:
to_drop_DoS = [2,3,4]
to_drop_Probe = [1,3,4]
to_drop_R2L = [1,2,4]
to_drop_U2R = [1,2,3]
DoS_df=newdf[~newdf['attack'].isin(to_drop_DoS)];
Probe_df=newdf[~newdf['attack'].isin(to_drop_Probe)];
R2L_df=newdf[~newdf['attack'].isin(to_drop_R2L)];
U2R_df=newdf[~newdf['attack'].isin(to_drop_U2R)];

#test
DoS_df_test=newdf_test[~newdf_test['attack'].isin(to_drop_DoS)];
Probe_df_test=newdf_test[~newdf_test['attack'].isin(to_drop_Probe)];
R2L_df_test=newdf_test[~newdf_test['attack'].isin(to_drop_R2L)];
U2R_df_test=newdf_test[~newdf_test['attack'].isin(to_drop_U2R)];
print('Train:')
print('Dimensions of DoS:' ,DoS_df.shape)
print('Dimensions of Probe:' ,Probe_df.shape)
print('Dimensions of R2L:' ,R2L_df.shape)
print('Dimensions of U2R:' ,U2R_df.shape)
print('Test:')
print('Dimensions of DoS:' ,DoS_df_test.shape)
print('Dimensions of Probe:' ,Probe_df_test.shape)
print('Dimensions of R2L:' ,R2L_df_test.shape)
print('Dimensions of U2R:' ,U2R_df_test.shape)

Train:
Dimensions of DoS: (113269, 124)
Dimensions of Probe: (78998, 124)
Dimensions of R2L: (68337, 124)
Dimensions of U2R: (67394, 124)
Test:
Dimensions of DoS: (17170, 124)
Dimensions of Probe: (12132, 124)
Dimensions of R2L: (12596, 124)
Dimensions of U2R: (9778, 124)


In [223]:
# Split dataframes into X & Y
# assign X as a dataframe of feautures and Y as a series of outcome variables
X_DoS = DoS_df.drop('attack', axis=1)
Y_DoS = DoS_df.attack
X_Probe = Probe_df.drop('attack', axis=1)
Y_Probe = Probe_df.attack
X_R2L = R2L_df.drop('attack', axis=1)
Y_R2L = R2L_df.attack
X_U2R = U2R_df.drop('attack', axis=1)
Y_U2R = U2R_df.attack
# test set
X_DoS_test = DoS_df_test.drop('attack', axis=1)
Y_DoS_test = DoS_df_test.attack
X_Probe_test = Probe_df_test.drop('attack', axis=1)
Y_Probe_test = Probe_df_test.attack
X_R2L_test = R2L_df_test.drop('attack', axis=1)
Y_R2L_test = R2L_df_test.attack
X_U2R_test = U2R_df_test.drop('attack', axis=1)
Y_U2R_test = U2R_df_test.attack

In [224]:
colNames=list(X_Probe)
colNames_test=list(X_Probe_test)

In [225]:
from sklearn import preprocessing

# Feature scaling - fit on training data, transform both train and test
scaler = preprocessing.StandardScaler().fit(X_Probe)
X_Probe = scaler.transform(X_Probe)
X_Probe_test = scaler.transform(X_Probe_test)

In [226]:
# Verify standardization results
print("Standard deviation check (should be ~1.0):")
print(X_Probe.std(axis=0)[:10])  # Show first 10 features

Standard deviation check (should be ~1.0):
[1. 1. 1. 1. 0. 1. 1. 1. 1. 1.]


In [227]:
Y_Probe.shape

(78998,)

In [228]:
X_Probe.shape

(78998, 123)

In [229]:
from sklearn.feature_selection import SelectPercentile, f_classif
np.seterr(divide='ignore', invalid='ignore');
selector=SelectPercentile(f_classif, percentile=10)
X_new = selector.fit_transform(X_Probe,Y_Probe)
X_new.shape

/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 4 16] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)


(78998, 13)

In [230]:
true=selector.get_support()
newcolindex=[i for i, x in enumerate(true) if x]
newcolname=list( colNames[i] for i in newcolindex )
newcolname

['logged_in',
 'rerror_rate',
 'srv_rerror_rate',
 'dst_host_srv_count',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_rerror_rate',
 'level',
 'Protocol_type_icmp',
 'service_eco_i',
 'service_private',
 'flag_SF']

In [231]:
from sklearn.feature_selection import RFE
clf = DecisionTreeClassifier(random_state=0)
rfe = RFE(estimator=clf, n_features_to_select=13, step=1)
rfe.fit(X_Probe, Y_Probe)
X_rfe=rfe.transform(X_Probe)
true=rfe.support_
rfecolindex=[i for i, x in enumerate(true) if x]
rfecolname=list(colNames[i] for i in rfecolindex)

In [232]:
# Display selected features for DoS classification
print('Features selected for Probe classification:')
for feature in rfecolname:
    print(f"  - {feature}")

Features selected for Probe classification:
  - src_bytes
  - dst_bytes
  - rerror_rate
  - dst_host_same_srv_rate
  - dst_host_diff_srv_rate
  - dst_host_same_src_port_rate
  - dst_host_rerror_rate
  - level
  - service_finger
  - service_ftp_data
  - service_http
  - service_private
  - service_smtp


In [233]:
print(X_rfe.shape)

(78998, 13)


In [234]:
# Train decision tree classifier
clf = DecisionTreeClassifier(random_state=0)

# Fit classifier
clf.fit(X_Probe, Y_Probe)

DecisionTreeClassifier(random_state=0)

In [235]:
# Train classifier with RFE-selected features

clf_rfe = DecisionTreeClassifier(random_state=0)
clf_rfe.fit(X_rfe, Y_Probe)

DecisionTreeClassifier(random_state=0)

In [236]:
X_Probe_test.shape


(12132, 123)

In [237]:
Y_Probe_test.shape

(12132,)

In [238]:
from sklearn.model_selection import cross_val_score
from sklearn import metrics

print("Evaluation Methodology Comparison:")
print("="*50)

# Cross-validation on training data (demonstrates inflated scores)
print("Cross-Validation on Training Data:")
accuracy = cross_val_score(clf, X_Probe, Y_Probe, cv=10, scoring='accuracy')
print("   CV Accuracy: %0.5f (+/- %0.5f)" % (accuracy.mean(), accuracy.std() * 2))

print("\nTrain-Test Evaluation:")
# Proper evaluation on test set 
clf.fit(X_Probe, Y_Probe)
Y_pred = clf.predict(X_Probe_test)
print("   Test Accuracy:", metrics.accuracy_score(Y_Probe_test, Y_pred))
print("   Test Precision:", metrics.precision_score(Y_Probe_test, Y_pred, pos_label=2))
print("   Test Recall:", metrics.recall_score(Y_Probe_test, Y_pred, pos_label=2))
print("   Test F1:", metrics.f1_score(Y_Probe_test, Y_pred, pos_label=2))


Evaluation Methodology Comparison:
Cross-Validation on Training Data:
   CV Accuracy: 0.99935 (+/- 0.00056)

Train-Test Evaluation:
   Test Accuracy: 0.9481536432575008
   Test Precision: 0.9155844155844156
   Test Recall: 0.815365551425031
   Test F1: 0.8625737382564999


In [239]:
X_test2 = X_Probe_test[:, rfecolindex]

Y_pred2 = clf_rfe.predict(X_test2)

# Confusion matrix
print(pd.crosstab(Y_Probe_test, Y_pred2, rownames=['Actual attacks'], colnames=['Predicted attacks']))

# Final evaluation on test set (no CV here)
print("Test Accuracy:", metrics.accuracy_score(Y_Probe_test, Y_pred2))
print("Test Precision:", metrics.precision_score(Y_Probe_test, Y_pred2, pos_label=2))
print("Test Recall:", metrics.recall_score(Y_Probe_test, Y_pred2, pos_label=2))
print("Test F1:", metrics.f1_score(Y_Probe_test, Y_pred2, pos_label=2))


Predicted attacks     0     2
Actual attacks               
0                  9447   264
2                   754  1667
Test Accuracy: 0.9160896801846357
Test Precision: 0.8632832729155878
Test Recall: 0.6885584469227591
Test F1: 0.7660845588235294


In [240]:
# RFE Feature Selection Evaluation
print("RFE-based Feature Selection Performance:")
print("="*50)

# Train model with RFE-selected features
clf_rfe.fit(X_Probe[:, rfecolindex], Y_Probe)
Y_pred_rfe = clf_rfe.predict(X_Probe_test[:, rfecolindex])

print("Test Performance with RFE Features:")
print(f"Test Accuracy: {metrics.accuracy_score(Y_Probe_test, Y_pred_rfe):.5f}")
print(f"Test Precision: {metrics.precision_score(Y_Probe_test, Y_pred_rfe, pos_label=2):.5f}")
print(f"Test Recall: {metrics.recall_score(Y_Probe_test, Y_pred_rfe, pos_label=2):.5f}")
print(f"Test F1-Score: {metrics.f1_score(Y_Probe_test, Y_pred_rfe, pos_label=2):.5f}")


RFE-based Feature Selection Performance:
Test Performance with RFE Features:
Test Accuracy: 0.91609
Test Precision: 0.86328
Test Recall: 0.68856
Test F1-Score: 0.76608


In [241]:
X = np.array(X_Probe)
X_test = np.array(X_Probe_test)
#cnn-input
X = np.reshape(X, (X.shape[0], X.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
#lstm-input

In [242]:
Y = np.array(Y_Probe)
Y_test = np.array(Y_Probe_test)

In [243]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint
from keras.optimizers import Adam
import tensorflow as tf

In [255]:
# Focal Loss Implementation for Class Imbalance
import tensorflow as tf
from tensorflow.keras import backend as K

def focal_loss(gamma=2.0, alpha=0.25):
    """
    Focal Loss implementation for addressing class imbalance
    
    Args:
        gamma (float): Focusing parameter to down-weight easy examples
        alpha (float): Weighting parameter for rare class (positive class)
    
    Returns:
        loss function
    """
    def focal_loss_fixed(y_true, y_pred):
        # Ensure y_pred is in valid range [epsilon, 1-epsilon]
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
        
        # Calculate cross entropy
        ce_loss = -y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred)
        
        # Calculate p_t
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        
        # Calculate alpha_t
        alpha_t = y_true * alpha + (1 - y_true) * (1 - alpha)
        
        # Calculate focal loss
        focal_loss = alpha_t * K.pow(1 - p_t, gamma) * ce_loss
        
        return K.mean(focal_loss)
    
    return focal_loss_fixed

# Calculate class weights for alpha parameter
from sklearn.utils.class_weight import compute_class_weight

# Calculate class distribution for DoS data
class_weights = compute_class_weight('balanced', classes=np.unique(Y), y=Y)
print(f"Class distribution:")
print(f"  Normal (0): {np.sum(Y == 0)} samples")
print(f"  Attack (2): {np.sum(Y == 2)} samples")
print(f"Class weights: {dict(zip(np.unique(Y), class_weights))}")

# Set alpha based on class distribution (weight for positive/minority class)
alpha_focal = class_weights[1] / (class_weights[0] + class_weights[1])
print(f"Focal Loss alpha parameter: {alpha_focal:.3f}")

# Create focal loss function with calculated alpha
focal_loss_fn = focal_loss(gamma=2.0, alpha=alpha_focal)

Class distribution:
  Normal (0): 67342 samples
  Attack (2): 11656 samples
Class weights: {0: 0.5865433162068249, 2: 3.3887268359643103}
Focal Loss alpha parameter: 0.852


In [256]:
import os

# Define a learning rate schedule
initial_learning_rate = 0.001
def lr_schedule(epoch):
    return initial_learning_rate * (0.1 ** (epoch // 10))

output_dir = '/kaggle/working/'
model_dir = os.path.join(output_dir, 'models')
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, 'best_cnn_lstm_dos_model.h5')

print(f"Model will be saved to: {model_path}")

my_callbacks = [
    ModelCheckpoint(
        filepath=model_path, 
        verbose=1, 
        save_best_only=True, 
        monitor='val_accuracy', 
        mode='max'
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1,
        min_delta=0.001
    ),
    LearningRateScheduler(lr_schedule, verbose=1)
]

Model will be saved to: /kaggle/working/models/best_cnn_lstm_dos_model.h5


In [ ]:
# CNN-LSTM Model Implementation
model = Sequential()

# Convolutional layers with L2 regularization
model.add(Conv1D(64, 3, padding="same", input_shape=(X_Probe.shape[1], 1), activation='relu', kernel_regularizer=l2(0.01)))
model.add(Conv1D(64, 3, padding="same", activation='relu', kernel_regularizer=l2(0.01)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(128, 3, padding="same", activation='relu', kernel_regularizer=l2(0.001)))
model.add(Conv1D(128, 3, padding="same", activation='relu', kernel_regularizer=l2(0.001)))
model.add(MaxPooling1D(pool_size=2))

# Batch normalization
model.add(BatchNormalization())

# LSTM layer with regularization and dropout
model.add(LSTM(units=100, return_sequences=False, dropout=0.1, kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(units=1, activation='sigmoid', kernel_regularizer=l2(0.01)))

# Model compilation with Focal Loss
opt = Adam(learning_rate=initial_learning_rate)

model.compile(
    loss=focal_loss_fn, 
    optimizer=opt, 
    metrics=[
        'accuracy',
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.AUC(name='auc')
    ]
)
# Proper validation strategy using train-validation split
from sklearn.model_selection import train_test_split

print("Validation Strategy: Stratified train-validation split")
print("Test set reserved for final evaluation only")

# Split training data (80/20)
X_train, X_val, y_train, y_val = train_test_split(
    X, Y, 
    test_size=0.2, 
    stratify=Y,
    random_state=42
)

print(f"Training samples: {X_train.shape[0]}")
print(f"Validation samples: {X_val.shape[0]}")
print(f"Test samples: {X_DoS_test.shape[0]}")

# Environment-independent callbacks
my_callbacks_proper = [
    ModelCheckpoint(
        filepath=model_path, 
        verbose=1, 
        save_best_only=True, 
        monitor='val_loss',
        mode='min'
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1,
        min_delta=0.001
    ),
    LearningRateScheduler(lr_schedule, verbose=1)
]

# Model training
print(f"\nTraining CNN-LSTM model...")
print(f"Model will be saved to: {model_path}")
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=my_callbacks_proper
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Validation Strategy: Stratified train-validation split
Test set reserved for final evaluation only
Training samples: 63198
Validation samples: 15800
Test samples: 17170

Training CNN-LSTM model...
Model will be saved to: /kaggle/working/models/best_cnn_lstm_dos_model.h5

Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/50
984/988 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5982 - auc: 0.8901 - loss: -1.7812 - precision: 0.3733 - recall: 0.9806
Epoch 1: val_loss improved from inf to -3.17479, saving model to /kaggle/working/models/best_cnn_lstm_dos_model.h5
988/988 ━━━━━━━━━━━━━━━━━━━━ 16s 11ms/step - accuracy: 0.5987 - auc: 0.8903 - loss: -1.7854 - precision: 0.3738 - recall: 0.9806 - val_accuracy: 0.7608 - val_auc: 0.9795 - val_loss: -3.1748 - val_precision: 0.6168 - val_recall: 1.0000 - learning_rate: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/50
988/988 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7896 - auc: 0.9741 - lo

In [258]:
y_pred = model.predict(X_Probe_test)

380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [259]:
y_pred1 = []
for i in y_pred:
  if i>=0.5:
    y_pred1.append(2)
  else:
    y_pred1.append(0)

In [260]:
y_pred = np.array(y_pred1)
y_pred

array([0, 2, 2, ..., 0, 0, 2])

In [261]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(Y_Probe_test,y_pred))

              precision    recall  f1-score   support

           0       0.98      0.95      0.97      9711
           2       0.82      0.94      0.88      2421

    accuracy                           0.95     12132
   macro avg       0.90      0.94      0.92     12132
weighted avg       0.95      0.95      0.95     12132

